# IN4080: obligatory assignment 1 (Autumn 2024)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, September 15 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [1]:
# this is my first version in which i use re.findall and works quite good
from typing import List
import re

txt = "Pierre, who works at NR, also teaches at UiO."

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""
    
    # Implement here your basic tokenisation
    
    x = re.findall(r'\b\w+\b|[.,:;!|?()"\'\[\]\-]', text)
    return print('tokens:\t', len(x),'\ntypes:\t', len(set(x)))

basic_tokenize(txt)

tokens:	 12 
types:	 10


In [2]:
# here we have the solution of the exercise in which i use re.split. It basically require less text but the final output is the same.
from typing import List
import re

txt = "Pierre, who works at NR, also teaches at UiO."

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""
    
    # Implement here your basic tokenisation
   
    tokens = re.split(r'(\W)', text)
    tokens = [token for token in tokens if token.strip()]
    print('tokens:\t', len(tokens))
    print('types:\t', len(set(tokens)))

tokens = basic_tokenize(txt)

tokens:	 12
types:	 10


In [3]:
# different attempts to get the solution
import re
txt = "Pierre, who works at NR, also teaches at UiO."
y = re.findall(r'\b\w+\b|[.,:;!?()"\'\[\]\-]', txt)
print(y)
print(len(y))

['Pierre', ',', 'who', 'works', 'at', 'NR', ',', 'also', 'teaches', 'at', 'UiO', '.']
12


We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [4]:
with open('ndt_test_lm.txt', 'r', encoding='utf-8') as file:
    ndt_test = file.read()

basic_tokenize(ndt_test)

# first attempt: I try to open the file with pandas but using the .csv the tokenization was a mass. Open is the best solution

tokens:	 259391
types:	 30021


We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [5]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

In [6]:
with open('ndt_train_lm.txt', 'r', encoding='utf-8') as file:
    ndt_train_lm = file.read()

with open('ndt_test_lm.txt', 'r', encoding='utf-8') as file:
    ndt_test_lm = file.read()

In [7]:
# learning the tokenizer
ndt_training_lm = BPETokenizer(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000)

# apply the tokenizer
ndt_testing_lm = list(ndt_training_lm(ndt_test_lm))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226096 [00:00<?, ?it/s]

In [8]:
print('tokens:\t\t\t\t', len(ndt_testing_lm))
print('types (distinct subwords):\t', len(set(ndt_testing_lm)))

tokens:				 386099
types (distinct subwords):	 4408


In [9]:
print("vocabulary:\t", ndt_training_lm.vocab[:50])

vocabulary:	 ['\x00', '\x01', '\x02', '\x03', '\x04', '\x05', '\x06', '\x07', '\x08', '\t', '\n', '\x0b', '\x0c', '\r', '\x0e', '\x0f', '\x10', '\x11', '\x12', '\x13', '\x14', '\x15', '\x16', '\x17', '\x18', '\x19', '\x1a', '\x1b', '\x1c', '\x1d', '\x1e', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1']


In [10]:
from collections import Counter

token_counts = Counter(list(ndt_training_lm(ndt_test_lm)))
print("frequencies:\t", token_counts)

print("merge pairs:\t", ndt_training_lm.merge_pairs)

print("vocab dim:\t", len(ndt_training_lm.vocab))

  0%|          | 0/226096 [00:00<?, ?it/s]

frequencies:	 Counter({'i ': 6942, 'og ': 5835, 'er ': 5341, 'en ': 5240, 'det ': 3845, 'som ': 3734, 'å ': 3593, 'på ': 3531, 'til ': 3251, 'at ': 2990, 'for ': 2935, 'av ': 2815, 'har ': 2698, 'de ': 2459, '- ': 2456, '. ': 2419, 'med ': 2414, ', ': 2359, 'ikke ': 2135, 'et ': 1782, 'om ': 1695, 'Det ': 1533, 'ene ': 1513, 'den ': 1473, 'var ': 1433, 's ': 1353, 'han ': 1224, 's': 1173, 'u': 1115, 't ': 1104, '-': 1095, 'for': 1091, 'fra ': 1076, 'er': 1065, 'er. ': 1065, 'jeg ': 1021, 'S': 1016, 'le': 992, 'e ': 973, 'kan ': 949, '"': 948, 'seg ': 942, 'er, ': 938, 'en. ': 916, 'vi ': 905, 'g': 892, 'an': 882, '| ': 879, 'sier ': 874, 'd': 849, 'men ': 842, 're': 812, 'ere ': 804, 'p': 789, 'K': 776, 't': 772, 'st': 759, 'så ': 756, 'de': 754, 'B': 751, 'b': 746, 'k': 743, 'ble ': 743, 'h': 738, 'a ': 737, 'der ': 735, 'te ': 733, 'en': 719, 'm': 711, 'Jeg ': 709, 'vil ': 707, 'F': 703, 'ut': 700, 'al': 698, 'ter ': 696, 'in': 695, 'etter ': 669, 'et. ': 661, 'd ': 659, 'te': 656, '

In [11]:
ndt_testing_lm

['La',
 'm ',
 'og ',
 'p',
 'ig',
 'g',
 'var ',
 'på ',
 'bry',
 'll',
 'up',
 'sm',
 'en',
 'y',
 'en ',
 '| ',
 'K',
 'am',
 'skj',
 'ell',
 ', ',
 'p',
 'ig',
 'g',
 'var ',
 'og ',
 'la',
 'mme',
 'fi',
 'let ',
 'sto ',
 'på ',
 'men',
 'y',
 'en ',
 'under ',
 'den ',
 'kong',
 'el',
 'ige ',
 'g',
 'al',
 'la',
 'mid',
 'dagen. ',
 'Og ',
 'til ',
 'dess',
 'ert',
 ': ',
 'Par',
 'fa',
 'it ',
 'à',
 ' ',
 'la ',
 'M',
 'ette',
 '-',
 'Mar',
 'it',
 '. ',
 'For',
 'retten ',
 'ly',
 'der ',
 'nav',
 'net ',
 '"',
 'Co',
 'qu',
 'il',
 'les ',
 'S',
 't. ',
 'J',
 'ac',
 'qu',
 'es ',
 'P',
 'r',
 'in',
 'ce ',
 'de ',
 'Nor',
 've',
 'ge',
 '", ',
 'som ',
 'er ',
 'gr',
 'il',
 'let ',
 'kam',
 'skj',
 'ell ',
 'på ',
 'norsk ',
 'spe',
 'ke',
 'sk',
 'in',
 'ke ',
 '- ',
 'med ',
 'trø',
 'ff',
 'el',
 'ho',
 'nn',
 'ing',
 'vin',
 'a',
 'ig',
 'rette',
 ', ',
 'ru',
 'c',
 'c',
 'o',
 'las',
 'al',
 'at ',
 'og ',
 'ris',
 'te',
 'de ',
 'g',
 'res',
 'skar',
 'kj',
 'er',


In [12]:
# the numbers of tokens and subwords are different in my own code and in the function implemented because of the byte pair encoding, in my function
# I only split words (take them as token)

__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

In [13]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm
import re

class BPETokenizer_mk2:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s" % train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str, int], n: int = 200) -> List[Tuple[str, str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords) - 1):
                byte_pair = (subwords[i], subwords[i + 1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input: str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword

    def tokenize_word(self, word: str) -> List[str]:
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""
        
        # Define expression pattern to treat punctuation as separate tokens
        pattern = re.compile(r'(\w+|[^\w\s])')
        
        # Split the word into a list of subword units, THIS IS THE MOST IMPORTANTE CHANGE OF THE MK2 VERSION OF THE TOKENIZER
        split_word = re.findall(pattern, word)
        
        # Initialize lists
        splits = []
        for part in split_word:
            splits.extend(list(part))
            splits.append(' ')  # ADD ADDITIONAL SEPARATOR TO DENOTE BOUNDARIES WITH DIFFERENT PARTS (check this part later for improv)

        while len(splits) >= 3:  # include separator + punctuation + character
            # xtraction of consecutive subword pairs, avoiding the separator
            pairs = [(splits[i], splits[i + 1]) for i in range(len(splits) - 1) 
                    if splits[i] != ' ' and splits[i + 1] != ' ' and re.match(r'\w', splits[i]) and re.match(r'\w', splits[i + 1])]

            if not pairs:
                break

            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs: # merge the two subwords
                i = 0
                while i < len(splits) - 1:
                    if (splits[i], splits[i + 1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i + 2:]
                        break
                    i += 1
            else:
                break
        
        # Remove the separator before returning
        splits = [s for s in splits if s != ' ']
        return splits

In [14]:
# the problem in this class was inside the tokenisation function, so i made some little improvemnt to try to solve it. Below I made some test to verify this

In [15]:
ndt_training_lm = BPETokenizer_mk2(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000)
ndt_testing_lm = list(ndt_training_lm(ndt_test_lm))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226096 [00:00<?, ?it/s]

In [16]:
print('tokens:\t\t\t\t', len(ndt_testing_lm))
print('types (distinct subwords):\t', len(set(ndt_testing_lm)))

tokens:				 408844
types (distinct subwords):	 2647


In [17]:
# check if 

In [18]:
text = 'Og då blir vi også tvinga til fornying, trur eg. '
list(ndt_training_lm(text))

# learning the tokenizer
ndt_training_lm = BPETokenizer_mk2(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000)

# apply the tokenizer
ndt_testing_lm = list(ndt_training_lm(text))

  0%|          | 0/10 [00:00<?, ?it/s]

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
print("vocabulary:\t", ndt_testing_lm[:50])

vocabulary:	 ['Og', 'då', 'blir', 'vi', 'også', 'tv', 'inga', 'til', 'forny', 'ing', ',', 'trur', 'eg', '.']


In [20]:
text.split()

['Og', 'då', 'blir', 'vi', 'også', 'tvinga', 'til', 'fornying,', 'trur', 'eg.']

In [ ]:
# as we can see the tokenizator work and split correctly the N-Grams and the punctuation

In [21]:
print('tokens:\t\t\t\t', len(ndt_testing_lm))
print('types (distinct subwords):\t', len(set(ndt_testing_lm)))

tokens:				 14
types (distinct subwords):	 14


__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published earlier this year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

1. **Why can't LLMs spell words correctly?**
The LLMs can struggle with spelling because they operate on subword tokens rather than individual characters or entire words. The tokenization split words into smaller pieces. If the model doesnt recognize the full word it might output incorrect spelling because it is reconstructiong the word from subword tokens and not because it knows it as a whole.

2. **Why is LLM worse at non-English languages (e.g., Japanese)?**
this is mainly due to the fact that LLMs struggle with the tokenization scheme designed for alphabet-based languages. An example are non-segmented languages like chinese and japanese (they dont use spaces) or character diversity which significantly increases the vocabulary size and makes harder for models to train.

3. **Why did my LLM abruptly halt when it sees the string "<|endoftext|>"?**
This is a special token used by models like GPT-2 to signify the end of a sequence or document. It can lead to unexpected early termination if the token is present in the middle of text or code by accident.

4. **Why did GPT-2 have more than necessary trouble coding in Python?** This can lead to inefficient or confusing token splits, making it harder for the model to generate syntactically correct or meaningful code

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [22]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()
 

    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [23]:
from collections import defaultdict, Counter

def __init__(self, training_corpus_file: str, tokenizer: BPETokenizer, ngram_size: int = 2, alpha_smoothing: float = 0.1):
    """Initialize the N-gram model with:
    - a file path to a training corpus to estimate the N-gram probabilities
    - an already learned BPE tokenizer
    - an N-gram size
    - a smoothing parameter (Laplace smoothing)
    """
    LanguageModel.__init__(self, tokenizer)
    self.ngram_size = ngram_size

    # Define a simple backoff distribution (uniform distribution)
    self.default_distrib = {token: 1/len(tokenizer.vocab) for token in tokenizer.vocab}

    # Dictionary mapping contexts (tuples of preceding words) to probability distributions
    self.ngram_probs = defaultdict(Counter)
    context_counts = defaultdict(Counter)
    
    # iteration corpus
    with open(training_corpus_file, 'r') as fd:
        for line in fd:
            tokens = ["<s>"] + list(self.tokenizer(line.strip())) + ["</s>"]
            
            # Create n-grams and context for each token
            for i in range(len(tokens) - ngram_size + 1):
                context = tuple(tokens[i:i + ngram_size - 1])
                token = tokens[i + ngram_size - 1]
                context_counts[context][token] += 1
    
    # Smoothed probabilities part
    vocab_size = len(self.tokenizer.vocab)
    
    for context, token_counts in context_counts.items():
        total_count = sum(token_counts.values()) + alpha_smoothing * vocab_size
        for token, count in token_counts.items():
            self.ngram_probs[context][token] = (count + alpha_smoothing) / total_count # This is Laplace smoothing
        
        # take only small probs
        for token in self.tokenizer.vocab:
            if token not in self.ngram_probs[context]:
                self.ngram_probs[context][token] = alpha_smoothing / total_count

setattr(NGramLanguageModel, '__init__', __init__) # --> this replace the func in the previus class 

__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [24]:
model = NGramLanguageModel(training_corpus_file='ndt_train_lm.txt', tokenizer=BPETokenizer(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/872824 [00:00<?, ?it/s]

In [25]:
# Read the test file and calculate perplexity
with open('ndt_test_lm.txt', 'r') as test_fd:
    test_text = test_fd.read()
    perplexity = model.get_perplexity(test_text)
    print(f"Test Perplexity: {perplexity}")

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/386099 [00:00<?, ?it/s]

Test Perplexity: 193.0345419254361


In [26]:
# Test Perplexity: 193.0345419254361
# now we try to compare this model with my version of the class BPETokenizer_mk2 and try to compare the perpelxity

In [41]:
model_mk2 = NGramLanguageModel(training_corpus_file='ndt_train_lm.txt', tokenizer=BPETokenizer_mk2(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/872824 [00:00<?, ?it/s]

In [42]:
# Read the test file and calculate perplexity
with open('ndt_test_lm.txt', 'r') as test_fd:
    test_text = test_fd.read()
    perplexity = model_mk2.get_perplexity(test_text)
    print(f"Test Perplexity: {perplexity}")

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/408844 [00:00<?, ?it/s]

Test Perplexity: 163.84116726365735


In [ ]:
# As we can see my implementation has low perplexity then the standard one. Good job!!!! <3

__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [27]:
# initialize tokenizer
tokenizer = BPETokenizer_mk2(train_corpus_file='ndt_train_class.txt')

Counting word occurrences in corpus ndt_train_class.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

In [28]:
type(tokenizer)

__main__.BPETokenizer_mk2

In [29]:
import numpy as np

# load Training set
sentences_train = []
labels_train = []

with open('ndt_train_class.txt') as file:
   for line in file:
      sent, label = line.strip().split('\t') # splitting the sentences and labels
      sentences_train.append(sent)
      labels_train.append(1 if label == 'nno' else 0)
      
# design matrix (N x V)
N_train = len(sentences_train)
V = len(tokenizer.vocab)
X_train = np.zeros((N_train, V))

# tokenization of design matrix --> we are creating a giant sparse matrix that is computationaly easy to use!
for i, sentence in enumerate(sentences_train):
    subwords = tokenizer.tokenize_word(sentence)
    for subword in subwords:
        if subword in tokenizer.vocab:
            X_train[i, tokenizer.vocab.index(subword)] = 1  # Set presence of subword
            
# converting labels in array for computation
y_train = np.array(labels_train) # --> ofc is a dicotomic vector

In [30]:
display(X_train)
display(y_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

array([1, 1, 1, ..., 0, 0, 0])

In [31]:
# load Test set --> copy and paste of the code above because the design matrix to build is the same.
sentences_test = []
labels_test = []

with open('ndt_test_class.txt') as file:
   for line in file:
      sent, label = line.strip().split('\t') # splitting the sentences and labels
      sentences_test.append(sent)
      labels_test.append(1 if label == 'nno' else 0)
      
# design matrix (N x V)
N_test = len(sentences_test)
V = len(tokenizer.vocab)
X_test = np.zeros((N_test, V))

# tokenization of design matrix
for i, sentence in enumerate(sentences_test):
    subwords = tokenizer.tokenize_word(sentence)
    for subword in subwords:
        if subword in tokenizer.vocab:
            X_test[i, tokenizer.vocab.index(subword)] = 1  # Set presence of subword
            
# converting labels in array for computation
y_test = np.array(labels_test)

__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [32]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(y_pred)

[1 1 1 ... 0 0 0]


In [ ]:
# we have the output of our model, but we are wworking with a logistic regression and we can investigate the probability of belonging to a class
# or to another

In [33]:
y_proba = model.predict_proba(X_test)
print(y_proba)

[[0.00247698 0.99752302]
 [0.04758486 0.95241514]
 [0.00389488 0.99610512]
 ...
 [0.93986683 0.06013317]
 [0.99896663 0.00103337]
 [0.80859823 0.19140177]]


__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'accuracy:\t {accuracy:.3f}')
print(f'precision:\t {precision:.3f}')
print(f'recall:\t\t {recall:.3f}')

accuracy:	 0.961
precision:	 0.974
recall:		 0.946


In [ ]:
# we have really good performance values!

__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [35]:
# first we check the size of our coefficient vector
coefficients = model.coef_
print(len(model.coef_[0]))

5056


In [36]:
# find the 5 subwords that contribute the most
top_nynorsk = np.argsort(coefficients[0])[-5:]
top_bokmal = np.argsort(coefficients[0])[:5]

In [37]:
print(top_nynorsk)
print(top_bokmal)

[484 493 578 517 656]
[684 545 491 640 686]


In [38]:
tokenizer.vocab[656]

'ikkje'

In [39]:
# Top 5 subwords that contribute the most in Nynorsk
for i in top_nynorsk:
   print(tokenizer.vocab[i])

ein
dei
frå
eit
ikkje


In [40]:
# Top 5 subwords that contribute the most in Bokmal
for i in top_bokmal:
   print(tokenizer.vocab[i])

sier
jeg
ikke
ble
Jeg


In [ ]:
# I dont speak Norwegian at all, but i check the values with the .txt files that we use and the weights make sanse. (Hopefully)